# Análise das Perguntas SQL com Python

## Localização de chamados do 1746
#### Utilize a tabela de Chamados do 1746 e a tabela de Bairros do Rio de Janeiro para as perguntas de 1-5.

1. Quantos chamados foram abertos no dia 01/04/2023?
2. Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?
3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?
4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?
5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?


## Chamados do 1746 em grandes eventos
#### Utilize a tabela de Chamados do 1746 e a tabela de Ocupação Hoteleira em Grandes Eventos no Rio para as perguntas de 6-10. Para todas as perguntas considere o subtipo de chamado "Perturbação do sossego".

6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?
7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).
8. Quantos chamados desse subtipo foram abertos em cada evento?
9. Qual evento teve a maior média diária de chamados abertos desse subtipo?
10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

##### Importante: a tabela de Chamados do 1746 possui mais de 10M de linhas. Evite fazer consultas exploratórias na tabela sem um filtro ou limite de linhas para economizar sua cota no BigQuery!

## Imports

In [7]:
import pandas as pd

# Dados

In [11]:
ocup = pd.read_csv('datasets/ocupacao.csv')
bairro = pd.read_csv('datasets/bairro.csv')
chamado = pd.read_parquet('datasets/chamado.parquet')


In [71]:
ocup.head()

,ano,data_inicial,data_final,evento,taxa_ocupacao
0,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
1,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
2,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184
3,08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451


In [76]:
# Função para gerar a lista de datas
def gerar_lista_datas(row):
    return pd.date_range(start=row['data_inicial'], end=row['data_final']).strftime('%Y-%m-%d').tolist()

# Aplicando a função em cada linha do DataFrame
#datas = ocup.apply(gerar_lista_datas, axis=1)
ocup['periodo'] = ocup.apply(gerar_lista_datas, axis=1)
ocup

,ano,data_inicial,data_final,evento,taxa_ocupacao,periodo
0,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554,"[2023-02-18, 2023-02-19, 2023-02-20, 2023-02-21]"
1,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251,"[2022-12-30, 2022-12-31, 2023-01-01]"
2,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184,"[2022-09-02, 2022-09-03, 2022-09-04]"
3,08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451,"[2022-09-08, 2022-09-09, 2022-09-10, 2022-09-11]"


In [103]:
df_explodido = ocup.explode('periodo')
df_explodido

,ano,data_inicial,data_final,evento,taxa_ocupacao,periodo
0,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554,2023-02-18
0,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554,2023-02-19
0,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554,2023-02-20
0,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554,2023-02-21
1,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251,2022-12-30
1,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251,2022-12-31
1,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251,2023-01-01
2,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184,2022-09-02
2,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184,2022-09-03
2,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184,2022-09-04


## Trativas

In [108]:
# Para fazer o merge dos dataframes bairro + chamado é preciso converter o tipo da coluna id_bairro
bairro['id_bairro'] = bairro['id_bairro'].apply(lambda x: str(x)) # converter o id_bairro de bairro para str porque possui menos registros no dataframe

# Merge(join) de chamado + bairro
cham_bair = pd.merge(chamado, bairro, how='left', on='id_bairro')

# Coluna de data
cham_bair['data_inicio'] = pd.to_datetime(cham_bair['data_inicio'])
cham_bair['data_inicio'] = cham_bair['data_inicio'].apply(lambda x: x.strftime('%Y-%m-%d'))

# Merge cham_bair + ocup
# Função para gerar a lista de datas
def gerar_lista_datas(row):
    return pd.date_range(start=row['data_inicial'], end=row['data_final']).strftime('%Y-%m-%d').tolist() # essa função analisa as datas de inicio e fim e retorna uma lista com todas as datas do evento

# Aplicando a função em cada linha do DataFrame
ocup['periodo'] = ocup.apply(gerar_lista_datas, axis=1) # agora o dataframe ocup possui uma coluna com listas dos dias de cada evento

# Transformando o dataframe para que cada data da coluna 'periodo' seja uma linha
df_explodido = ocup.explode('periodo') # A função explode transforma listas em várias linhas, mantendo as demais colunas
df_explodido = df_explodido[['periodo', 'evento', 'taxa_ocupacao']] # Filtra o dataframe
df_explodido.columns = ['data_inicio', 'evento', 'taxa_ocupacao'] # muda o nome da coluna para ser possível fazer o merge

# Faz o join do dataframe cham_bair com o df_explodido
merged = pd.merge(cham_bair, df_explodido, how='left', on='data_inicio') 

In [110]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1634336 entries, 0 to 1634335
Data columns (total 26 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   id_chamado                   1634336 non-null  object 
 1   data_inicio                  1634336 non-null  object 
 2   id_bairro                    1562182 non-null  object 
 3   nome_unidade_organizacional  1634336 non-null  object 
 4   categoria                    1634336 non-null  object 
 5   tipo                         1634336 non-null  object 
 6   subtipo                      1634336 non-null  object 
 7   status                       1634336 non-null  object 
 8   longitude                    1192576 non-null  float64
 9   latitude                     1192576 non-null  float64
 10  situacao                     1634336 non-null  object 
 11  tipo_situacao                1634336 non-null  object 
 12  reclamacoes                  1634336 non-n

In [41]:
cham_bair.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1634336 entries, 0 to 1634335
Data columns (total 24 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   id_chamado                   1634336 non-null  object 
 1   data_inicio                  1634336 non-null  object 
 2   id_bairro                    1562182 non-null  object 
 3   nome_unidade_organizacional  1634336 non-null  object 
 4   categoria                    1634336 non-null  object 
 5   tipo                         1634336 non-null  object 
 6   subtipo                      1634336 non-null  object 
 7   status                       1634336 non-null  object 
 8   longitude                    1192576 non-null  float64
 9   latitude                     1192576 non-null  float64
 10  situacao                     1634336 non-null  object 
 11  tipo_situacao                1634336 non-null  object 
 12  reclamacoes                  1634336 non-n

## Respostas

1. **Quantos chamados foram abertos no dia 01/04/2023?**

In [47]:
abril_01 = cham_bair[cham_bair['data_inicio'] == '2023-04-01']
len(abril_01)

1756

2. **Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?**

In [50]:
abril_01['tipo'].value_counts().head(1)

tipo
Estacionamento irregular    366
Name: count, dtype: int64

3. **Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?**

In [55]:
#abril_01.groupby('nome').agg({'id_chamado':'count'}).sort_values('id_chamado', ascending=False).head(3)
abril_01['nome'].value_counts().head(3)

nome
Campo Grande       113
Tijuca              89
Barra da Tijuca     59
Name: count, dtype: int64

4. **Qual o nome da subprefeitura com mais chamados abertos nesse dia?**

In [56]:
abril_01['subprefeitura'].value_counts().head(1)

subprefeitura
Zona Norte    510
Name: count, dtype: int64

5. **Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?**

In [62]:
print(f'Existem {len(cham_bair[cham_bair['nome'].isnull()])} chamados abertos nesse dia que não foram associados a bairro nem subprefeituras')
print('Isso acontece pois não há informações sobre o local no registro do chamado. Não há id do bairro, nem coordenadas e nem qualquer informação que possibilite identificar de onde vem o chamado.')

Existem 72154 chamados abertos nesse dia que não foram associados a bairro nem subprefeituras
Isso acontece pois não há informações sobre o local no registro do chamado. Não há id do bairro, nem coordenadas e nem qualquer informação que possibilite identificar de onde vem o chamado.


In [60]:
cham_bair[cham_bair['nome'].isnull()]['status'].unique()

array(['Sem possibilidade de atendimento', 'Fechado com informação',
       'Não constatado', 'Fechado com providências',
       'Fechado com solução', 'Pendente', 'Aberto', 'Cancelado',
       'Em Andamento'], dtype=object)

Inclusive, analisando os status dos mesmos podemos perceber que nenhum dos chamados pode ser atendido

6. **Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?**

In [70]:
# O dataframe já está previamente filtrado com o período do dia 01/01/202 e 31/12/2023(considerando que as extremidades sejam o primeiro e último dia deste período)
print(f'Foram abertos {sum(cham_bair['subtipo'] == 'Perturbação do sossego')} chamados com o subtipo "Perturbação do sossego" dentro deste período.')

Foram abertos 42830 chamados com o subtipo "Perturbação do sossego" dentro deste período.


7. **Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).**

In [120]:
eventos = merged[merged['evento'].isin(['Reveillon', 'Carnaval','Rock in Rio'])]
eventos[eventos['subtipo'] == 'Perturbação do sossego']

,id_chamado,data_inicio,id_bairro,nome_unidade_organizacional,categoria,tipo,subtipo,status,longitude,latitude,...,nome_regiao_planejamento,id_regiao_administrativa,nome_regiao_administrativa,subprefeitura,area,perimetro,geometry_wkt,geometry,evento,taxa_ocupacao
7750,17663234,2022-09-03,28,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Fechado com solução,-43.224021,-22.962225,...,Zona Sul,6.0,Lagoa,Zona Sul,2.689204e+06,12263.131623,POLYGON ((-43.204182360353876 -22.955817747731...,"POLYGON((-43.2041823603539 -22.9558177477313, ...",Rock in Rio,0.8184
7751,17684216,2022-09-10,42,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Fechado com solução,-43.265079,-22.852406,...,Ramos,10.0,Ramos,Zona Norte,3.689833e+06,12231.009936,POLYGON ((-43.256280603077975 -22.837866685590...,"POLYGON((-43.256280603078 -22.8378666855908, -...",Rock in Rio,0.9451
7760,17682475,2022-09-09,64,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Fechado com solução,-43.286791,-22.891278,...,Méier,13.0,Meier,Zona Norte,1.012636e+06,6163.388130,POLYGON ((-43.2848445736727 -22.88758600816928...,"POLYGON((-43.2848445736727 -22.8875860081693, ...",Rock in Rio,0.9451
7762,17662396,2022-09-03,59,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Fechado com solução,-43.257869,-22.905025,...,Méier,13.0,Meier,Zona Norte,9.281085e+05,5342.536959,POLYGON ((-43.25220181097268 -22.8961093505744...,"POLYGON((-43.2522018109727 -22.8961093505744, ...",Rock in Rio,0.8184
7764,17684395,2022-09-11,43,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Fechado com solução,-43.280718,-22.844154,...,Penha,11.0,Penha,Zona Norte,5.811323e+06,13667.794012,POLYGON ((-43.25648886199383 -22.8336276062704...,"POLYGON((-43.2564888619938 -22.8336276062704, ...",Rock in Rio,0.9451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605279,18078334,2022-12-30,17,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Sem possibilidade de atendimento,-43.181432,-22.933223,...,Zona Sul,4.0,Botafogo,Zona Sul,2.493507e+06,10537.128736,POLYGON ((-43.18148564548288 -22.9295123629833...,"POLYGON((-43.1814856454829 -22.9295123629834, ...",Reveillon,0.9251
1605295,18075939,2022-12-30,25,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Sem possibilidade de atendimento,-43.203012,-22.984636,...,Zona Sul,6.0,Lagoa,Zona Sul,3.084907e+06,23487.557826,MULTIPOLYGON (((-43.194871617225914 -22.977457...,MULTIPOLYGON(((-43.1948716172259 -22.977457772...,Reveillon,0.9251
1605353,18078245,2022-12-30,141,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Sem possibilidade de atendimento,-43.478031,-22.882524,...,Bangu,17.0,Bangu,Grande Bangu,3.596620e+07,46529.035222,POLYGON ((-43.487172994385624 -22.848805326665...,"POLYGON((-43.4871729943856 -22.8488053266659, ...",Reveillon,0.9251
1605405,18077790,2022-12-30,25,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Sem possibilidade de atendimento,NaN,NaN,...,Zona Sul,6.0,Lagoa,Zona Sul,3.084907e+06,23487.557826,MULTIPOLYGON (((-43.194871617225914 -22.977457...,MULTIPOLYGON(((-43.1948716172259 -22.977457772...,Reveillon,0.9251


8. **Quantos chamados desse subtipo foram abertos em cada evento?**

In [122]:
eventos[eventos['subtipo'] == 'Perturbação do sossego'].groupby('evento').agg({'id_chamado':'count'})

,id_chamado
evento,
Carnaval,241
Reveillon,139
Rock in Rio,834


9. **Qual evento teve a maior média diária de chamados abertos desse subtipo?**

In [143]:
eventos_2 = eventos[eventos['subtipo'] == 'Perturbação do sossego'].groupby(['evento','data_inicio']).agg({'id_chamado':'count'}).reset_index()
eventos_2.groupby('evento').agg({'id_chamado':'mean'})

,id_chamado
evento,
Carnaval,60.250000
Reveillon,46.333333
Rock in Rio,119.142857


O Rock in Rio teve uma média diária de chamados abertos do subtipo Perturbação ao sossego bem maior do que os outros eventos

10. **Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.**

In [150]:
df2 = cham_bair[cham_bair['subtipo'] == 'Perturbação do sossego'].groupby('data_inicio').agg({'id_chamado':'count'})
#df2['id_chamado'].mean()
df2.mean()

id_chamado    61.982634
dtype: float64

O Rock em Rio definitivamente possui uma média de chamados para Perturbação ao sossego bem maior do que a média geral(quase o dobro).
<br>Sendo uma média diária de 119 chamados deste subtipo só no Rock in Rio contra uma média de 62 chamados diários ao longo dos anos 2022 e 2023 juntos.
<br> O Carnaval possui uma média de chamados diários muito próxima do normal, enquanto é possível notar uma queda nessa estimativa durante o Reveillon. 